## Лінійна та поліноміальна регресія для ціноутворення гарбузів - Урок 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Інфографіка від Дасані Мадіпаллі</figcaption>


#### Вступ

До цього моменту ви досліджували, що таке регресія, використовуючи вибіркові дані з набору даних про ціноутворення гарбузів, який ми будемо використовувати протягом цього уроку. Ви також візуалізували ці дані за допомогою `ggplot2`. 💪

Тепер ви готові заглибитися в регресію для машинного навчання. У цьому уроці ви дізнаєтеся більше про два типи регресії: *основну лінійну регресію* та *поліноміальну регресію*, а також про деякі математичні основи цих методів.

> Протягом цього курсу ми припускаємо мінімальні знання математики та прагнемо зробити матеріал доступним для студентів з інших галузей, тому звертайте увагу на примітки, 🧮 підказки, діаграми та інші навчальні інструменти, які допоможуть у розумінні.

#### Підготовка

Нагадаємо, ви завантажуєте ці дані, щоб ставити запитання.

-   Коли найкращий час для купівлі гарбузів?

-   Яку ціну можна очікувати за ящик мініатюрних гарбузів?

-   Чи варто купувати їх у кошиках на пів-бушеля або в коробках на 1 1/9 бушеля? Давайте продовжимо досліджувати ці дані.

У попередньому уроці ви створили `tibble` (сучасне переосмислення фрейму даних) і заповнили його частиною оригінального набору даних, стандартизуючи ціни за бушель. Однак, таким чином, ви змогли зібрати лише близько 400 точок даних і лише для осінніх місяців. Можливо, ми зможемо отримати трохи більше деталей про природу даних, якщо очистимо їх краще? Побачимо... 🕵️‍♀️

Для цього завдання нам знадобляться наступні пакети:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) — це [колекція пакетів для R](https://www.tidyverse.org/packages), яка робить науку про дані швидшою, простішою та цікавішою!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) — це [фреймворк](https://www.tidymodels.org/packages/) для моделювання та машинного навчання, що складається з колекції пакетів.

-   `janitor`: [пакет janitor](https://github.com/sfirke/janitor) надає прості інструменти для аналізу та очищення "брудних" даних.

-   `corrplot`: [пакет corrplot](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) забезпечує візуальний інструмент для дослідження матриці кореляції, який підтримує автоматичне впорядкування змінних для виявлення прихованих закономірностей між змінними.

Ви можете встановити їх за допомогою:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Скрипт нижче перевіряє, чи є у вас необхідні пакети для виконання цього модуля, і встановлює їх у разі відсутності.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Ми пізніше завантажимо ці чудові пакети та зробимо їх доступними у нашій поточній R-сесії. (Це лише для ілюстрації, `pacman::p_load()` вже зробив це за вас)

## 1. Лінія лінійної регресії

Як ви дізналися в Уроці 1, мета вправи з лінійної регресії полягає в тому, щоб побудувати *лінію* *найкращого* *підходу*, щоб:

-   **Показати взаємозв’язки змінних**. Показати взаємозв’язок між змінними.

-   **Робити прогнози**. Робити точні прогнози щодо того, де нова точка даних може розташуватися відносно цієї лінії.

Для побудови такого типу лінії ми використовуємо статистичний метод, який називається **регресія найменших квадратів**. Термін `найменші квадрати` означає, що всі точки даних навколо лінії регресії підносяться до квадрату, а потім додаються. Ідеально, щоб кінцева сума була якомога меншою, оскільки ми прагнемо до низької кількості помилок, або `найменших квадратів`. Таким чином, лінія найкращого підходу — це лінія, яка дає нам найнижче значення суми квадратів помилок — звідси назва *регресія найменших квадратів*.

Ми робимо це, оскільки хочемо змоделювати лінію, яка має найменшу сумарну відстань від усіх наших точок даних. Ми також підносимо терміни до квадрату перед додаванням, оскільки нас цікавить їх величина, а не напрямок.

> **🧮 Покажіть мені математику**
>
> Ця лінія, яка називається *лінією найкращого підходу*, може бути виражена [рівнянням](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` — це '`пояснювальна змінна` або `предиктор`'. `Y` — це '`залежна змінна` або `результат`'. Нахил лінії — це `b`, а `a` — це точка перетину з віссю Y, яка відповідає значенню `Y`, коли `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "нахил = $y/x$")
    Інфографіка від Jen Looper
>
> Спочатку обчисліть нахил `b`.
>
> Іншими словами, і звертаючись до початкового питання про дані гарбузів: "прогнозуйте ціну гарбуза за бушель залежно від місяця", `X` буде означати ціну, а `Y` — місяць продажу.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Інфографіка від Jen Looper
> 
> Обчисліть значення Y. Якщо ви платите близько \$4, це має бути квітень!
>
> Математика, яка обчислює лінію, повинна показувати нахил лінії, який також залежить від точки перетину, або де `Y` розташований, коли `X = 0`.
>
> Ви можете ознайомитися з методом обчислення цих значень на веб-сайті [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Також відвідайте [цей калькулятор найменших квадратів](https://www.mathsisfun.com/data/least-squares-calculator.html), щоб побачити, як значення чисел впливають на лінію.

Не так вже й страшно, правда? 🤓

#### Кореляція

Ще один термін, який потрібно зрозуміти, — це **коефіцієнт кореляції** між заданими змінними X і Y. Використовуючи діаграму розсіювання, ви можете швидко візуалізувати цей коефіцієнт. Графік із точками даних, розташованими в акуратній лінії, має високу кореляцію, але графік із точками даних, розкиданими хаотично між X і Y, має низьку кореляцію.

Хороша модель лінійної регресії буде тією, яка має високий (ближче до 1, ніж до 0) коефіцієнт кореляції, використовуючи метод регресії найменших квадратів із лінією регресії.


## **2. Танці з даними: створення датафрейму для моделювання**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Ілюстрація від @allison_horst</figcaption>


<!--![Ілюстрація від \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Завантажте необхідні бібліотеки та набір даних. Перетворіть дані у датафрейм, що містить підмножину даних:

-   Отримайте лише гарбузи, ціна яких вказана за бушель

-   Перетворіть дату на місяць

-   Розрахуйте ціну як середнє значення між високою та низькою ціною

-   Перетворіть ціну, щоб вона відображала вартість за кількість у бушелях

> Ми розглянули ці кроки у [попередньому уроці](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

У дусі чистої пригоди давайте дослідимо [`janitor package`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor), який надає прості функції для аналізу та очищення брудних даних. Наприклад, давайте поглянемо на назви стовпців у наших даних:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Ми можемо зробити краще. Давайте зробимо ці назви стовпців `friendR`, перетворивши їх у формат [snake_case](https://en.wikipedia.org/wiki/Snake_case) за допомогою `janitor::clean_names`. Щоб дізнатися більше про цю функцію: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Багато tidyR 🧹! Тепер танець з даними за допомогою `dplyr`, як у попередньому уроці! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Молодець!👌 Тепер у вас є чистий, впорядкований набір даних, на основі якого можна створити нову модель регресії!

Як щодо діаграми розсіювання?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Діаграма розсіювання нагадує нам, що у нас є дані лише за місяці з серпня по грудень. Ймовірно, нам потрібно більше даних, щоб мати змогу робити висновки лінійним способом.

Давайте ще раз подивимося на наші дані для моделювання:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

А що, якби ми хотіли передбачити `price` гарбуза на основі стовпців `city` або `package`, які є текстовими? Або навіть простіше, як би ми могли знайти кореляцію (яка вимагає, щоб обидва її вхідні значення були числовими) між, наприклад, `package` і `price`? 🤷🤷

Моделі машинного навчання найкраще працюють із числовими ознаками, а не текстовими значеннями, тому зазвичай потрібно перетворювати категоріальні ознаки на числові представлення.

Це означає, що нам потрібно знайти спосіб переформатувати наші предиктори, щоб зробити їх зручнішими для використання моделлю, процес, який називається `feature engineering`.


## 3. Підготовка даних для моделювання за допомогою recipes 👩‍🍳👨‍🍳

Дії, які змінюють формат значень предикторів, щоб зробити їх більш зручними для використання моделлю, називаються `інженерією ознак`.

Різні моделі мають різні вимоги до попередньої обробки даних. Наприклад, метод найменших квадратів вимагає `кодування категоріальних змінних`, таких як місяць, сорт і назва міста. Це просто передбачає `перетворення` стовпця з `категоріальними значеннями` в один або кілька `числових стовпців`, які замінюють оригінальний.

Наприклад, припустимо, що ваші дані містять таку категоріальну ознаку:

|  місто   |
|:--------:|
|  Денвер  |
|  Найробі |
|   Токіо  |

Ви можете застосувати *порядкове кодування*, щоб замінити кожну категорію унікальним цілим числом, ось так:

| місто |
|:-----:|
|   0   |
|   1   |
|   2   |

І саме це ми зробимо з нашими даними!

У цьому розділі ми ознайомимося з іншим чудовим пакетом Tidymodels: [recipes](https://tidymodels.github.io/recipes/) - який створений для того, щоб допомогти вам підготувати дані **перед** тренуванням моделі. У своїй основі рецепт — це об'єкт, який визначає, які кроки слід застосувати до набору даних, щоб підготувати його до моделювання.

Тепер давайте створимо рецепт, який підготує наші дані для моделювання, замінюючи унікальним цілим числом всі спостереження у стовпцях предикторів:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Чудово! 👏 Ми щойно створили наш перший рецепт, який визначає результат (ціна) та відповідні предиктори, а також те, що всі стовпці предикторів мають бути закодовані у набір цілих чисел 🙌! Давайте швидко розберемо це:

- Виклик `recipe()` із формулою повідомляє рецепту *ролі* змінних, використовуючи дані `new_pumpkins` як еталон. Наприклад, стовпцю `price` було призначено роль `outcome`, тоді як решті стовпців було призначено роль `predictor`.

- `step_integer(all_predictors(), zero_based = TRUE)` визначає, що всі предиктори мають бути перетворені у набір цілих чисел, починаючи з нуля.

Ми впевнені, що у вас можуть виникати думки на кшталт: "Це так круто!! Але що, якщо мені потрібно переконатися, що рецепти роблять саме те, що я очікую? 🤔"

Це чудова думка! Розумієте, як тільки ваш рецепт визначено, ви можете оцінити параметри, необхідні для фактичної попередньої обробки даних, а потім витягти оброблені дані. Зазвичай вам не потрібно робити це, коли ви використовуєте Tidymodels (ми побачимо звичайну практику за хвилину -> `workflows`), але це може бути корисним, якщо ви хочете провести перевірку, щоб переконатися, що рецепти працюють так, як ви очікуєте.

Для цього вам знадобляться ще два дієслова: `prep()` та `bake()`, і, як завжди, наші маленькі друзі з R від [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) допоможуть вам краще зрозуміти це!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Ілюстрація від @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): оцінює необхідні параметри з навчального набору, які потім можуть бути застосовані до інших наборів даних. Наприклад, для заданого стовпця предиктора, яке спостереження буде призначено цілому числу 0, 1, 2 тощо.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): бере підготовлений рецепт і застосовує операції до будь-якого набору даних.

Отже, давайте підготуємо та застосуємо наші рецепти, щоб дійсно переконатися, що "під капотом" стовпці предикторів спочатку будуть закодовані перед тим, як модель буде побудована.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Ура! 🥳 Оброблені дані `baked_pumpkins` мають всі свої предиктори закодовані, що підтверджує, що кроки попередньої обробки, визначені як наш рецепт, працюють як очікувалося. Це робить дані складнішими для читання, але набагато зрозумілішими для Tidymodels! Приділіть трохи часу, щоб з'ясувати, яке спостереження було зіставлено з відповідним цілим числом.

Також варто зазначити, що `baked_pumpkins` — це датафрейм, на якому ми можемо виконувати обчислення.

Наприклад, давайте спробуємо знайти хорошу кореляцію між двома точками ваших даних, щоб потенційно побудувати хороший предиктивний модель. Для цього ми використаємо функцію `cor()`. Введіть `?cor()`, щоб дізнатися більше про цю функцію.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Як виявилося, між Містом і Ціною існує лише слабка кореляція. Однак між Пакетом і його Ціною кореляція трохи сильніша. Це логічно, чи не так? Зазвичай, чим більша коробка з продуктами, тим вища ціна.

Поки ми цим займаємося, давайте також спробуємо візуалізувати матрицю кореляції для всіх стовпців, використовуючи пакет `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Набагато краще.

Гарне питання, яке тепер можна поставити до цих даних: '`Яку ціну я можу очікувати за певний пакет гарбузів?`' Давайте розберемося!

> Примітка: Коли ви **`bake()`** підготовлений рецепт **`pumpkins_prep`** з **`new_data = NULL`**, ви отримуєте оброблені (тобто закодовані) навчальні дані. Якщо у вас є інший набір даних, наприклад тестовий набір, і ви хочете побачити, як рецепт його попередньо обробить, просто виконайте **`bake`** для **`pumpkins_prep`** з **`new_data = test_set`**.

## 4. Побудова моделі лінійної регресії

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Інфографіка від Дасані Мадіпаллі</figcaption>


<!--![Інфографіка від Дасані Мадіпаллі](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Тепер, коли ми створили рецепт і фактично підтвердили, що дані будуть належним чином попередньо оброблені, давайте побудуємо модель регресії, щоб відповісти на запитання: `Яку ціну я можу очікувати для заданого пакунка гарбуза?`

#### Навчання моделі лінійної регресії за допомогою навчального набору

Як ви, можливо, вже здогадалися, стовпець *price* є змінною `результату`, тоді як стовпець *package* є змінною `предиктором`.

Для цього ми спочатку розділимо дані таким чином, щоб 80% потрапило до навчального набору, а 20% — до тестового набору, потім визначимо рецепт, який закодує стовпець предиктора у набір цілих чисел, а потім створимо специфікацію моделі. Ми не будемо готувати та випікати наш рецепт, оскільки вже знаємо, що він попередньо обробить дані належним чином.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Чудова робота! Тепер, коли у нас є рецепт і специфікація моделі, потрібно знайти спосіб об'єднати їх в один об'єкт, який спочатку буде виконувати попередню обробку даних (prep+bake за лаштунками), навчати модель на попередньо оброблених даних, а також дозволяти потенційні дії після обробки. Як вам така впевненість!🤩

У Tidymodels цей зручний об'єкт називається [`workflow`](https://workflows.tidymodels.org/) і зручно зберігає ваші компоненти моделювання! Це те, що ми назвали б *пайплайнами* у *Python*.

Отже, давайте об'єднаємо все в workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

👌 До того ж, робочий процес можна налаштувати/тренувати майже так само, як і модель.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

З результатів моделі ми можемо побачити коефіцієнти, отримані під час навчання. Вони представляють коефіцієнти лінії найкращого наближення, яка забезпечує найменшу загальну помилку між фактичними та прогнозованими змінними.

#### Оцінка продуктивності моделі за допомогою тестового набору

Час перевірити, як модель справляється 📏! Як це зробити?

Тепер, коли ми навчили модель, ми можемо використовувати її для створення прогнозів для `test_set` за допомогою `parsnip::predict()`. Потім ми можемо порівняти ці прогнози з фактичними значеннями міток, щоб оцінити, наскільки добре (або не дуже!) працює модель.

Давайте почнемо з створення прогнозів для тестового набору, а потім об'єднаємо ці стовпці з тестовим набором.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Так, ви щойно натренували модель і використали її для прогнозування! 🔮 Чи вона хороша? Давайте оцінемо продуктивність моделі!

У Tidymodels ми робимо це за допомогою `yardstick::metrics()`! Для лінійної регресії зосередимося на наступних метриках:

-   `Root Mean Square Error (RMSE)`: Квадратний корінь [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Це дає абсолютну метрику в тих самих одиницях, що й мітка (у цьому випадку ціна гарбуза). Чим менше значення, тим краща модель (у спрощеному сенсі це представляє середню величину, на яку прогнози помиляються!).

-   `Coefficient of Determination (зазвичай відомий як R-squared або R2)`: Відносна метрика, де чим більше значення, тим краще модель відповідає даним. По суті, ця метрика показує, яку частину варіації між прогнозованими та фактичними значеннями міток модель здатна пояснити.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Ось і продуктивність моделі. Давайте спробуємо отримати кращу оцінку, візуалізуючи точковий графік пакета та ціни, а потім використаємо зроблені прогнози, щоб накласти лінію найкращого підходу.

Це означає, що нам потрібно підготувати та обробити тестовий набір, щоб закодувати стовпець пакета, а потім об'єднати це з прогнозами, зробленими нашою моделлю.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Чудово! Як бачите, модель лінійної регресії не дуже добре узагальнює взаємозв'язок між пакунком і його відповідною ціною.

🎃 Вітаємо, ви щойно створили модель, яка може допомогти передбачити ціну кількох сортів гарбузів. Ваш святковий гарбузовий сад буде чудовим. Але, ймовірно, ви можете створити кращу модель!

## 5. Створіть модель поліноміальної регресії

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Інфографіка від Дасані Мадіпаллі</figcaption>


<!--![Інфографіка від Дасані Мадіпаллі](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Іноді наші дані можуть не мати лінійної залежності, але ми все одно хочемо передбачити результат. Поліноміальна регресія може допомогти нам робити прогнози для більш складних нелінійних залежностей.

Візьмемо, наприклад, залежність між упаковкою та ціною в нашому наборі даних про гарбузи. Хоча іноді між змінними існує лінійна залежність — чим більший гарбуз за об'ємом, тим вища ціна — іноді ці залежності не можна зобразити у вигляді площини або прямої лінії.

> ✅ Ось [декілька прикладів](https://online.stat.psu.edu/stat501/lesson/9/9.8) даних, які можуть використовувати поліноміальну регресію
>
> Погляньте ще раз на залежність між сортом і ціною на попередньому графіку. Чи здається, що цей розсіювальний графік обов’язково слід аналізувати за допомогою прямої лінії? Можливо, ні. У цьому випадку ви можете спробувати поліноміальну регресію.
>
> ✅ Поліноми — це математичні вирази, які можуть складатися з однієї або кількох змінних і коефіцієнтів

#### Навчання моделі поліноміальної регресії за допомогою навчального набору

Поліноміальна регресія створює *криву лінію*, щоб краще відповідати нелінійним даним.

Давайте перевіримо, чи поліноміальна модель буде краще виконувати прогнози. Ми будемо дотримуватися схожої процедури, як і раніше:

-   Створити рецепт, який визначає кроки попередньої обробки, які слід виконати над нашими даними, щоб підготувати їх до моделювання, тобто: кодування предикторів і обчислення поліномів ступеня *n*

-   Побудувати специфікацію моделі

-   Об'єднати рецепт і специфікацію моделі в робочий процес

-   Створити модель шляхом налаштування робочого процесу

-   Оцінити, наскільки добре модель працює на тестових даних

Давайте почнемо!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Оцінка продуктивності моделі

👏👏Ви створили поліноміальну модель, давайте зробимо прогнози на тестовому наборі!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Ура, давайте оцінимо, як модель виконала на test_set, використовуючи `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Значно краща продуктивність.

`rmse` зменшився приблизно з 7 до 3, що свідчить про зменшення помилки між фактичною ціною та прогнозованою ціною. Це можна *вільно* інтерпретувати як те, що в середньому неправильні прогнози помиляються приблизно на \$3. `rsq` зріс приблизно з 0.4 до 0.8.

Усі ці метрики свідчать про те, що поліноміальна модель працює значно краще, ніж лінійна модель. Чудова робота!

Давайте спробуємо це візуалізувати!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Ви можете побачити криву, яка краще відповідає вашим даним! 🤩

Ви можете зробити її більш плавною, передавши поліноміальну формулу до `geom_smooth` ось так:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Дуже схоже на плавну криву!🤩

Ось як можна зробити новий прогноз:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


Прогнозування за допомогою `поліноміальної моделі` дійсно має сенс, враховуючи діаграми розсіювання для `price` та `package`! І якщо ця модель краща за попередню, дивлячись на ті ж самі дані, вам потрібно врахувати бюджет для цих дорожчих гарбузів!

🏆 Чудова робота! Ви створили дві регресійні моделі за один урок. У фінальному розділі про регресію ви дізнаєтеся про логістичну регресію для визначення категорій.

## **🚀Виклик**

Перевірте кілька різних змінних у цьому ноутбуці, щоб побачити, як кореляція впливає на точність моделі.

## [**Тест після лекції**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Огляд і самостійне навчання**

У цьому уроці ми вивчили лінійну регресію. Є й інші важливі типи регресії. Прочитайте про методи Stepwise, Ridge, Lasso та Elasticnet. Хороший курс для подальшого вивчення — це [курс зі статистичного навчання Стенфорда](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Якщо ви хочете дізнатися більше про використання дивовижного фреймворку Tidymodels, ознайомтеся з наступними ресурсами:

-   Вебсайт Tidymodels: [Почніть з Tidymodels](https://www.tidymodels.org/start/)

-   Макс Кун і Джулія Сілдж, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **ДЯКУЄМО:**

[Елісон Хорст](https://twitter.com/allison_horst?lang=en) за створення дивовижних ілюстрацій, які роблять R більш доступним і цікавим. Знайдіть більше ілюстрацій у її [галереї](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критично важливої інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають у результаті використання цього перекладу.
